<a href="https://colab.research.google.com/github/shirleytingyen/lecture-ai-engineering/blob/master/day1/day1_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実践演習 Day 1：streamlitとFastAPIのデモ
このノートブックでは以下の内容を学習します。

- 必要なライブラリのインストールと環境設定
- Hugging Faceからモデルを用いたStreamlitのデモアプリ
- FastAPIとngrokを使用したAPIの公開方法

演習を始める前に、HuggingFaceとngrokのアカウントを作成し、
それぞれのAPIトークンを取得する必要があります。


演習の時間では、以下の3つのディレクトリを順に説明します。

1. 01_streamlit_UI
2. 02_streamlit_app
3. 03_FastAPI

2つ目や3つ目からでも始められる様にノートブックを作成しています。

復習の際にもこのノートブックを役立てていただければと思います。

### 注意事項
「02_streamlit_app」と「03_FastAPI」では、GPUを使用します。

これらを実行する際は、Google Colab画面上のメニューから「編集」→ 「ノートブックの設定」

「ハードウェアアクセラレーター」の項目の中から、「T4 GPU」を選択してください。

このノートブックのデフォルトは「CPU」になっています。

---

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 環境変数の設定（1~3共有）


GitHubから演習用のコードをCloneします。

In [30]:
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

fatal: destination path 'lecture-ai-engineering' already exists and is not an empty directory.


必要なAPIトークンを.envに設定します。

「lecture-ai-engineering/day1」の配下に、「.env_template」ファイルが存在しています。

隠しファイルのため表示されていない場合は、画面左側のある、目のアイコンの「隠しファイルの表示」ボタンを押してください。

「.env_template」のファイル名を「.env」に変更します。「.env」ファイルを開くと、以下のような中身になっています。


```
HUGGINGFACE_TOKEN="hf-********"
NGROK_TOKEN="********"
```
ダブルクオーテーションで囲まれた文字列をHuggingfaceのアクセストークンと、ngrokの認証トークンで書き変えてください。

それぞれのアカウントが作成済みであれば、以下のURLからそれぞれのトークンを取得できます。

- Huggingfaceのアクセストークン
https://huggingface.co/docs/hub/security-tokens

- ngrokの認証トークン
https://dashboard.ngrok.com/get-started/your-authtoken

書き換えたら、「.env」ファイルをローカルのPCにダウンロードしてください。

「01_streamlit_UI」から「02_streamlit_app」へ進む際に、CPUからGPUの利用に切り替えるため、セッションが一度切れてしまいます。

その際に、トークンを設定した「.env」ファイルは再作成することになるので、その手間を減らすために「.env」ファイルをダウンロードしておくと良いです。

「.env」ファイルを読み込み、環境変数として設定します。次のセルを実行し、最終的に「True」が表示されていればうまく読み込めています。

In [31]:
!pip install python-dotenv
from dotenv import load_dotenv, find_dotenv

%cd /content/lecture-ai-engineering/day1
load_dotenv(find_dotenv())

/content/lecture-ai-engineering/day1


True

# 01_streamlit_UI

ディレクトリ「01_streamlit_UI」に移動します。

In [32]:
%cd /content/lecture-ai-engineering/day1/01_streamlit_UI

/content/lecture-ai-engineering/day1/01_streamlit_UI


必要なライブラリをインストールします。

In [33]:
%%capture
!pip install -r requirements.txt

ngrokのトークンを使用して、認証を行います。

In [34]:
!ngrok authtoken $$NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


アプリを起動します。

In [35]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://184e-34-91-59-112.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.59.112:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/local/bin/streamlit", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1082, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1697, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1443, in invoke
    return ctx.invoke(self.callback, **ctx.pa

公開URLの後に記載されているURLにブラウザでアクセスすると、streamlitのUIが表示されます。

app.pyのコメントアウトされている箇所を編集することで、UIがどの様に変化するか確認してみましょう。

streamlitの公式ページには、ギャラリーページがあります。

streamlitを使うとpythonという一つの言語であっても、様々なUIを実現できることがわかると思います。

https://streamlit.io/gallery

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [36]:
from pyngrok import ngrok
ngrok.kill()

# 02_streamlit_app


ディレクトリ「02_streamlit_app」に移動します。

In [37]:
%cd /content/lecture-ai-engineering/day1/02_streamlit_app

/content/lecture-ai-engineering/day1/02_streamlit_app


必要なライブラリをインストールします。

In [38]:
%%capture
!pip install -r requirements.txt

In [39]:
# !pip install bitsandbytes==0.41.1
# !pip install torch==2.0.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# # !pip install transformers==4.31.0
# !pip install transformers==4.34.0 sentence-transformers==2.2.2 streamlit==1.24.0
# !pip install huggingface_hub==0.17.3

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [40]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


stramlitでHuggingfaceのトークン情報を扱うために、streamlit用の設定ファイル（.streamlit）を作成し、トークンの情報を格納します。

In [41]:
# .streamlit/secrets.toml ファイルを作成
import os
import toml

# 設定ファイルのディレクトリ確保
os.makedirs('.streamlit', exist_ok=True)

# 環境変数から取得したトークンを設定ファイルに書き込む
secrets = {
    "huggingface": {
        "token": os.environ.get("HUGGINGFACE_TOKEN", "")
    }
}

# 設定ファイルを書き込む
with open('.streamlit/secrets.toml', 'w') as f:
    toml.dump(secrets, f)

アプリを起動します。

02_streamlit_appでは、Huggingfaceからモデルをダウンロードするため、初回起動には2分程度時間がかかります。

この待ち時間を利用して、app.pyのコードを確認してみましょう。

In [42]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://88eb-34-91-59-112.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.59.112:8501

  Stopping...
  Stopping...


アプリケーションの機能としては、チャット機能や履歴閲覧があります。

これらの機能を実現するためには、StreamlitによるUI部分だけではなく、SQLiteを使用したチャット履歴の保存やLLMのモデルを呼び出した推論などの処理を組み合わせることで実現しています。

- **`app.py`**: アプリケーションのエントリーポイント。チャット機能、履歴閲覧、サンプルデータ管理のUIを提供します。
- **`ui.py`**: チャットページや履歴閲覧ページなど、アプリケーションのUIロジックを管理します。
- **`llm.py`**: LLMモデルのロードとテキスト生成を行うモジュール。
- **`database.py`**: SQLiteデータベースを使用してチャット履歴やフィードバックを保存・管理します。
- **`metrics.py`**: BLEUスコアやコサイン類似度など、回答の評価指標を計算するモジュール。
- **`data.py`**: サンプルデータの作成やデータベースの初期化を行うモジュール。
- **`config.py`**: アプリケーションの設定（モデル名やデータベースファイル名）を管理します。
- **`requirements.txt`**: このアプリケーションを実行するために必要なPythonパッケージ。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [43]:
from pyngrok import ngrok
ngrok.kill()

# 03_FastAPI

ディレクトリ「03_FastAPI」に移動します。

In [44]:
%cd /content/lecture-ai-engineering/day1/03_FastAPI

/content/lecture-ai-engineering/day1/03_FastAPI


必要なライブラリをインストールします。

In [45]:
%%capture
!pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [46]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


アプリを起動します。

「02_streamlit_app」から続けて「03_FastAPI」を実行している場合は、モデルのダウンロードが済んでいるため、すぐにサービスが立ち上がります。

「03_FastAPI」のみを実行している場合は、初回の起動時にモデルのダウンロードが始まるので、モデルのダウンロードが終わるまで数分間待ちましょう。

In [47]:
!python app.py

/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-04-22 10:48:10.701382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745318890.734581   54761 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745318890.747718   54761 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory 

FastAPIが起動すると、APIとクライアントが通信するためのURL（エンドポイント）が作られます。

URLが作られるのと合わせて、Swagger UIというWebインターフェースが作られます。

Swagger UIにアクセスすることで、APIの仕様を確認できたり、APIをテストすることができます。

Swagger UIを利用することで、APIを通してLLMを動かしてみましょう。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [48]:
from pyngrok import ngrok
ngrok.kill()

In [60]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   .gitignore
	deleted:    day1/.env_template
	new file:   day1/01_streamlit_UI/lecture-ai-engineering
	modified:   day1/02_streamlit_app/app.py
	modified:   day1/02_streamlit_app/config.py
	modified:   day1/02_streamlit_app/data.py
	modified:   day1/02_streamlit_app/database.py
	new file:   day1/02_streamlit_app/lecture-ai-engineering
	modified:   day1/02_streamlit_app/llm.py
	modified:   day1/02_streamlit_app/metrics.py
	modified:   day1/02_streamlit_app/requirements.txt
	modified:   day1/02_streamlit_app/ui.py
	new file:   day1/03_FastAPI/lecture-ai-engineering
	new file:   lecture-ai-engineering/day1/02_streamlit_app/app.py
	new file:   lecture-ai-engineering/day1/02_streamlit_app/config.py
	new file:   lecture-ai-engineering/day1/02_streamlit_app/data.py
	new file:   lecture-ai-engineering/day1/02_streamlit_app/database.py
	new fi

In [61]:
!git add .

In [63]:
!git config user.name "shirleytingyen"
!git config user.email "shirleywang840106@gmail.com"

In [64]:
!git commit -m "量子化と動的なモデル選択を追加する"

[master f1586d5] 量子化と動的なモデル選択を追加する
 22 files changed, 2631 insertions(+), 556 deletions(-)
 create mode 100644 .gitignore
 delete mode 100644 day1/.env_template
 create mode 160000 day1/01_streamlit_UI/lecture-ai-engineering
 rewrite day1/02_streamlit_app/app.py (96%)
 rewrite day1/02_streamlit_app/config.py (100%)
 rewrite day1/02_streamlit_app/data.py (98%)
 create mode 160000 day1/02_streamlit_app/lecture-ai-engineering
 rewrite day1/02_streamlit_app/llm.py (79%)
 rewrite day1/02_streamlit_app/metrics.py (91%)
 create mode 160000 day1/03_FastAPI/lecture-ai-engineering
 create mode 100644 lecture-ai-engineering/day1/02_streamlit_app/app.py
 create mode 100644 lecture-ai-engineering/day1/02_streamlit_app/config.py
 create mode 100644 lecture-ai-engineering/day1/02_streamlit_app/data.py
 create mode 100644 lecture-ai-engineering/day1/02_streamlit_app/database.py
 create mode 160000 lecture-ai-engineering/day1/02_streamlit_app/lecture-ai-engineering
 create mode 100644 lecture-ai-engine

In [65]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address
